# 1 - Data Collection - QPR (Posts)

In [1]:
# imports
import requests
import pandas as pd
import datetime as dt
import time
import praw
import numpy as np

In [2]:
base_url = 'https://api.pushshift.io/reddit/submission/search'

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
# Credit for the backbone of this function goes to Chris Joyce DSI 
# adjusted for personal requirement

def pull_reddit_posts(subreddit):
    
    # takes a subreddit name and pulls posts from 2011 to the current time in chunks of 500
    # prints the time taken for each chunk and the total number of posts pulled
    # returns a dataframe of posts with each row being a unique submission 
    
    posts = []

    
    print("this is subreddit", subreddit)
    # save URL based on current subreddit
    url = f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=500"
    # date to start as set to 2011
    start_utc = dt.datetime(2011, 1, 1).timestamp()
    print("start_utc", start_utc)
    while True:
        
        start_time = time.time()
        response = requests.get(url)
        # save json as data
        # if JSONDECODEError then pass the submission and return error message 
        try:
            data = response.json()
            print("Equipment data has been successfully retrieved.")
        except json.decoder.JSONDecodeError:
            print("There was a problem accessing the equipment data.")
        # if data key not in response, break loop
        if len(data['data']) == 0:
            break
        posts += data['data']
        print("posts length", len(posts))
        last_post = data['data'][-1]
        created_utc = last_post['created_utc']
        print("created_utc", created_utc)
        
        # if time passes the start time then break
        if created_utc < start_utc:
            break
        # if loop hasn't broken, set url for next pull passing created_utc as new 'before' parameter
        url = f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=500&before={created_utc}"
        end_time = time.time()
        print("total time taken this loop: ", end_time-start_time)
    df = pd.DataFrame(posts)
    return df

In [5]:
df_n = pull_reddit_posts('superhoops')

this is subreddit superhoops
start_utc 1293840000.0
Equipment data has been successfully retrieved.
posts length 500
created_utc 1641671788
total time taken this loop:  3.6767842769622803
Equipment data has been successfully retrieved.
posts length 1000
created_utc 1610376662
total time taken this loop:  3.1002390384674072
Equipment data has been successfully retrieved.
posts length 1500
created_utc 1576166154
total time taken this loop:  2.2192611694335938
Equipment data has been successfully retrieved.
posts length 2000
created_utc 1536757366
total time taken this loop:  2.7542529106140137
Equipment data has been successfully retrieved.
posts length 2500
created_utc 1502295869
total time taken this loop:  2.571800947189331
Equipment data has been successfully retrieved.
posts length 3000
created_utc 1475186629
total time taken this loop:  2.864793062210083
Equipment data has been successfully retrieved.
posts length 3500
created_utc 1455808745
total time taken this loop:  3.070982933

In [6]:
df_n.head()

,subreddit,selftext,author_fullname,gilded,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,thumbnail_height,top_awarded_type,hide_score,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,total_awards_received,media_embed,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,score,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,post_hint,content_categories,is_self,link_flair_type,wls,removed_by_category,author_flair_type,domain,allow_live_comments,suggested_sort,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,preview,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,removed_by,distinguished,subreddit_id,link_flair_background_color,id,is_robot_indexable,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,retrieved_utc,updated_utc,utc_datetime_str,poll_data,link_flair_template_id,crosspost_parent_list,crosspost_parent,is_gallery,media_metadata,gallery_data,author_cakeday,author_created_utc,retrieved_on,edited_on,removal_reason,approved_by,created,downs,likes,mod_reports,num_reports,post_categories,report_reasons,rte_mode,saved,selftext_html,ups,user_reports,brand_safe,approved_at_utc,banned_at_utc,from,from_id,from_kind
0,superhoops,,t2_knjfx,0.0,Balogun statement on the altercation with fans...,[],r/superhoops,False,6.0,None,140.0,NaN,True,False,dark,1.0,transparent,public,0.0,"{'content': '&lt;blockquote class=""twitter-vid...",140.0,318dcdec-2cdb-11eb-9af1-0ee629a70bc3,False,"{'type': 'twitter.com', 'oembed': {'provider_u...",False,False,NaN,"{'content': '&lt;blockquote class=""twitter-vid...",None,1,False,False,https://b.thumbs.redditmedia.com/rBqSXMxE7v6gw...,False,None,[],{},link,NaN,False,text,6.0,None,text,twitter.com,False,NaN,https://twitter.com/LeonBalogun/status/1642543...,NaN,False,True,True,False,False,{'images': [{'source': {'url': 'https://extern...,[],[],False,True,False,False,Scottish R's 🏴󠁧󠁢󠁳󠁣󠁴󠁿,[],NaN,None,t5_2swvo,,129pgft,True,Deadend_Friend,NaN,0,True,all_ads,False,False,dark,/r/superhoops/comments/129pgft/balogun_stateme...,all_ads,False,https://twitter.com/LeonBalogun/status/1642543...,2577.0,1680451313,0.0,"{'type': 'twitter.com', 'oembed': {'provider_u...",False,1680451327,1680451327,2023-04-02 16:01:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,superhoops,Sacked by Fernandes when ultimately he was the...,t2_frdzs,0.0,Irony if Warnock keeps Huddersfield up in plac...,[],r/superhoops,False,6.0,None,NaN,NaN,True,False,dark,1.0,transparent,public,0.0,{},NaN,60b296e6-2cdd-11eb-9077-0eaf553ab7a7,False,None,False,False,NaN,{},None,1,False,False,self,False,None,[],{},NaN,NaN,True,text,6.0,None,text,self.superhoops,False,NaN,NaN,NaN,False,True,True,False,False,NaN,[],[],False,True,False,False,England R's 🏴󠁧󠁢󠁥󠁮󠁧󠁿,[],NaN,None,t5_2swvo,,129d0fq,True,GaxZE,NaN,0,True,all_ads,False,False,dark,/r/superhoops/comments/129d0fq/irony_if_warnoc...,all_ads,False,https://www.reddit.com/r/superhoops/comments/1...,2576.0,1680418448,0.0,None,False,1680418465,1680418466,2023-04-02 06:54:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,superhoops,I thought he had some cheek slagging off the f...,t2_knjfx,0.0,Video of the incident with Balogun at full tim...,[],r/superhoops,False,6.0,None,140.0,NaN,True,False,dark,1.0,transparent,public,0.0,"{'content': '&lt;blockquote class=""twitter-vid...",140.0,318dcdec-2cdb-11eb-9af1-0ee629a70bc3,False,"{'type': 'twitter.com', 'oembed': {'provider_u

In [38]:
# empty selftext values filled with title text

df_n.selftext.replace('', np.nan, inplace = True)

df_n.selftext.fillna(df_n.title, inplace = True)

df_n.head(10)

,subreddit,selftext,author_fullname,gilded,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,thumbnail_height,top_awarded_type,hide_score,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,total_awards_received,media_embed,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,score,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,post_hint,content_categories,is_self,link_flair_type,wls,removed_by_category,author_flair_type,domain,allow_live_comments,suggested_sort,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,preview,all_awardings,awarders,media_only,can_gild,spoiler,locked,author_flair_text,treatment_tags,removed_by,distinguished,subreddit_id,link_flair_background_color,id,is_robot_indexable,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,retrieved_utc,updated_utc,utc_datetime_str,poll_data,link_flair_template_id,crosspost_parent_list,crosspost_parent,is_gallery,media_metadata,gallery_data,author_cakeday,author_created_utc,retrieved_on,edited_on,removal_reason,approved_by,created,downs,likes,mod_reports,num_reports,post_categories,report_reasons,rte_mode,saved,selftext_html,ups,user_reports,brand_safe,approved_at_utc,banned_at_utc,from,from_id,from_kind,length
0,superhoops,Balogun statement on the altercation with fans...,t2_knjfx,0.0,Balogun statement on the altercation with fans...,[],r/superhoops,False,6.0,None,140.0,NaN,True,False,dark,1.0,transparent,public,0.0,"{'content': '&lt;blockquote class=""twitter-vid...",140.0,318dcdec-2cdb-11eb-9af1-0ee629a70bc3,False,"{'type': 'twitter.com', 'oembed': {'provider_u...",False,False,NaN,"{'content': '&lt;blockquote class=""twitter-vid...",None,1,False,False,https://b.thumbs.redditmedia.com/rBqSXMxE7v6gw...,False,None,[],{},link,NaN,False,text,6.0,None,text,twitter.com,False,NaN,https://twitter.com/LeonBalogun/status/1642543...,NaN,False,True,True,False,False,{'images': [{'source': {'url': 'https://extern...,[],[],False,True,False,False,Scottish R's 🏴󠁧󠁢󠁳󠁣󠁴󠁿,[],NaN,None,t5_2swvo,,129pgft,True,Deadend_Friend,NaN,0,True,all_ads,False,False,dark,/r/superhoops/comments/129pgft/balogun_stateme...,all_ads,False,https://twitter.com/LeonBalogun/status/1642543...,2577.0,1680451313,0.0,"{'type': 'twitter.com', 'oembed': {'provider_u...",False,1680451327,1680451327,2023-04-02 16:01:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
1,superhoops,Sacked by Fernandes when ultimately he was the...,t2_frdzs,0.0,Irony if Warnock keeps Huddersfield up in plac...,[],r/superhoops,False,6.0,None,NaN,NaN,True,False,dark,1.0,transparent,public,0.0,{},NaN,60b296e6-2cdd-11eb-9077-0eaf553ab7a7,False,None,False,False,NaN,{},None,1,False,False,self,False,None,[],{},NaN,NaN,True,text,6.0,None,text,self.superhoops,False,NaN,NaN,NaN,False,True,True,False,False,NaN,[],[],False,True,False,False,England R's 🏴󠁧󠁢󠁥󠁮󠁧󠁿,[],NaN,None,t5_2swvo,,129d0fq,True,GaxZE,NaN,0,True,all_ads,False,False,dark,/r/superhoops/comments/129d0fq/irony_if_warnoc...,all_ads,False,https://www.reddit.com/r/superhoops/comments/1...,2576.0,1680418448,0.0,None,False,1680418465,1680418466,2023-04-02 06:54:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42
2,superhoops,I thought he had some cheek slagging off the f...,t2_knjfx,0.0,Video of the incident with Balogun at full tim...,[],r/superhoops,False,6.0,None,140.0,NaN,True,False,dark,1.0,transparent,public,0.0,"{'content': '&lt;blockquote class=""twitter-vid...",140.0,318dcdec-2cdb-11eb-9af1-0ee6

In [29]:
# posts longer than 10 words are kept
df_n['length'] = df_n.selftext.str.split().apply(lambda x: len(x))
long = df_n['length'] > 10
df_n = df_n[long]

In [30]:
df_n.length.min()

11

In [31]:
df_n.shape

(3303, 121)

In [32]:
df_n.selftext.value_counts(ascending = False)

2015/16 season tickets are now on sale - with prices frozen from our last @SkyBetChamp campaign                                                                                                                                                                                                                     2
QPR ace sparks war with West Ham striker: "Something doesn't seem right!"                                                                                                                                                                                                                                           2
Exclusive - Neil Warnock not in line for permanent Queens Park Rangers job, he tells talkSPORT                                                                                                                                                                                                                      2
QPR Director of Football Les Ferdinand watches on during the SSE Airtr

In [33]:
# columns to keep

to_keep = df_n[[
    'subreddit',
    'selftext',
    'score',
    'length',
    'num_comments',
    'utc_datetime_str',
    'id'
]]

In [34]:
df_posts = to_keep

In [35]:
# date time columns converted to format 'YYYY-MM-DD'

df_posts.utc_datetime_str = df_posts.utc_datetime_str.str[:10]

/var/folders/5h/p8nqld_13hg44lctlhg2ns8c0000gn/T/ipykernel_3975/3181375314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_posts.utc_datetime_str = df_posts.utc_datetime_str.str[:10]


In [ ]:
df_posts.head()

In [37]:
df_posts.num_comments.mean()

3.0647895852255527

In [19]:
# dataframe saved to datasets folder

# df_posts.to_csv('../datasets/qpr_posts.csv', index = False)